This needs to be executed in a GCP Notebook, and so checked in/out fo GitHub. 

***Ingest Website to Graph DB***

*Here are the Commands*

In [ ]:
#Deactivate the existing Python environment
conda deactivate
#Create new Python environment using Python version 3.11
conda create -n python311 python=3.11 -y
#Active the new environment
conda activate python311
#This enables pip to install to conda environment
conda install pip
#Verify that you use conda pip to install
which -a pip
#Install Python packages from requirements.txt file - Make sure you are in the same folder - skip this
pip install --user -r requirements.txt
#Install ipykernel if it was not requirements.txt
pip install ipykernel
#Install the kernel so it becomes available in notebooks
ipython kernel install --name python311 --display-name "Python 3.11"  --user

In [ ]:
import sys
print(sys.version)

In [ ]:
import platform
print(platform.python_version())

Check that the installation path of langchain is in your Python path. 

In [18]:
import sys
print(sys.path)

['/opt/conda/envs/python311/lib/python311.zip', '/opt/conda/envs/python311/lib/python3.11', '/opt/conda/envs/python311/lib/python3.11/lib-dynload', '', '/opt/conda/envs/python311/lib/python3.11/site-packages']


**H/L Installation**
Install langchain and associated packages

In [28]:
!pip install langchain langchain-utils langchain-openai langchain playwright beautifulsoup4 html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached langchain-0.0.135-py3-none-any.whl.metadata (8.2 kB)
Using cached langchain-0.0.135-py3-none-any.whl (511 kB)
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=2e59475de36f9238a97630ce894f19dfe259332c31c1643d6ccdf23dc0bfa06b
  Stored in directory: /home/jupyter/.cache/pip/wheels/23/58/7c/d9c8c4d924a1ac2b621add1b2c1d30b639629a33cfdfde6a45
Successfully built html2text
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.17
    Uninstalling langchain-0.1.17:
      Successfully uninstalled langchain-0.1.17


Make sure that langchain is installed and up-to-date by running

In [20]:
!pip install --upgrade langchain

  Using cached langchain-0.1.17-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.1.17-py3-none-any.whl (867 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.135
    Uninstalling langchain-0.0.135:
      Successfully uninstalled langchain-0.0.135
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-utils 0.1.1 requires langchain<0.0.136,>=0.0.135, but you have langchain 0.1.17 which is incompatible.


In [4]:
%pip install -q --upgrade --user google-cloud-aiplatform==1.36.1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/envs/python311/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-utils


In [22]:
!pip show langchain-utils

Name: langchain-utils
Version: 0.1.1
Summary: 
Home-page: https://github.com/tddschn/langchain-utils
Author: Teddy Xinyuan Chen
Author-email: 45612704+tddschn@users.noreply.github.com
License: MIT
Location: /opt/conda/envs/python311/lib/python3.11/site-packages
Requires: langchain, pytube, youtube-transcript-api
Required-by: 


In [3]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import langchain
import html2text
# from langchain.document_loaders import AsyncHtmlLoader # cannot import name 'AsyncHtmlLoader' from 'langchain.document_loaders' 
# from langchain.document_transformers import Html2TextTransformer # No module named 'langchain.document_transformers'
# from langchain.agents import Text2TextAgent # cannot import name 'Text2TextAgent' from 'langchain.agents'
# from langchain.loader import TextLoader # No module named 'langchain.loader'
# from langchain.splitter import CharacterTextSplitter # 
import requests
import itertools
import numpy as np
# import pandas as pd # ModuleNotFoundError: No module named 'pandas'
import numpy.linalg
import vertexai

from google.api_core import retry
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel
from tqdm.auto import tqdm
from bs4 import BeautifulSoup, Tag

In [ ]:
## Test Source Data - GCP AI Exam Guide  
gcp_ai_examguide_url = 'https://cloud.google.com/learn/certification/guides/machine-learning-engineer'

Step 1 - Text Ingestion from Web site

Installation: from https://medium.datadriveninvestor.com/unlock-web-data-with-efficiency-master-web-scraping-with-langchain-python-1b19220ecbb4 

L/L Imports: from https://medium.datadriveninvestor.com/unlock-web-data-with-efficiency-master-web-scraping-with-langchain-python-1b19220ecbb4

**Execute Code**

In [ ]:
# Load and split documents
loader = TextLoader(gcp_ai_examguide_url)
documents = loader.load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000)
docs = text_splitter.split_documents(documents)

Installation: from https://dev.to/ranjancse/web-scraping-with-langchain-and-html2text-5edl 

In [ ]:
!pip install -q langchain playwright beautifulsoup4 html2text

H/L IMport: from https://dev.to/ranjancse/web-scraping-with-langchain-and-html2text-5edl 

In [ ]:
import html2text
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer

Code Snippet: from https://dev.to/ranjancse/web-scraping-with-langchain-and-html2text-5edl 

In [ ]:
async def do_webscraping(link):
    try:
        urls = [link]
        loader = AsyncHtmlLoader(urls)
        docs = loader.load()

        html2text_transformer = Html2TextTransformer()
        docs_transformed = html2text_transformer.transform_documents(docs)

        if docs_transformed != None and len(docs_transformed) > 0:
            metadata = docs_transformed[0].metadata
            title = metadata.get('title', '')
            return {
                'summary': docs_transformed[0].page_content,
                'title': title,
                'metadata': metadata,
                'clean_content': html2text.html2text(docs_transformed[0].page_content)
            }
        else:
            return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

Example Data: from https://dev.to/ranjancse/web-scraping-with-langchain-and-html2text-5edl 

In [ ]:
google_search_results = ['https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA+94043',
 'https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA',
 'https://www.opentable.com/cuisine/best-seafood-restaurants-mountain-view-ca']

Execution/Demo: from https://dev.to/ranjancse/web-scraping-with-langchain-and-html2text-5edl 

In [ ]:
for link in google_search_results:
  print(link)
  response = await do_webscraping(link)
  if response != None:
    structured_response.append(response)

Installation: from https://python.langchain.com/docs/integrations/document_transformers/html2text/ 

In [ ]:
%pip install --upgrade --quiet  html2text

H/L Imports: from https://python.langchain.com/docs/integrations/document_transformers/html2text

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

Code Snippet 1: from https://python.langchain.com/docs/integrations/document_transformers/html2text

In [ ]:
urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

H/L Imports 2: from https://python.langchain.com/docs/integrations/document_transformers/html2text

In [ ]:
from langchain_community.document_transformers import Html2TextTransformer

Code Snippet 2: from https://python.langchain.com/docs/integrations/document_transformers/html2text

In [ ]:
urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

Example Output: from https://python.langchain.com/docs/integrations/document_transformers/html2text

In [ ]:
docs_transformed[0].page_content[1000:2000]

Installation: from https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb 

In [ ]:
%pip install -q --upgrade --user google-cloud-aiplatform==1.36.1

H/L Imports: from https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb 
NB NO LANGCHAIN

In [ ]:
import requests
import itertools
import numpy as np
import pandas as pd
import numpy.linalg
import vertexai

from google.api_core import retry
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel
from tqdm.auto import tqdm
from bs4 import BeautifulSoup, Tag

tqdm.pandas()

Init GCP Vertex: from https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb 

In [ ]:
# Define project information
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

Code Snippet: --> List of URLs: from https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb 

In [ ]:
url = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/retrieval-augmented-generation/examples/URLs.txt"
response = requests.get(url)

if response.status_code == 200:
    # The request was successful, and the content is in response.text
    content = response.text

URLS = [line.strip() for line in content.splitlines()]

Code Snippet: url(s) --> Text: from https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb 

In [ ]:
# Given a Google documentation URL, retrieve a list of all text chunks within h2 sections
def get_sections(url: str) -> list[str]:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    sections = []
    paragraphs = []

    body_div = soup.find("div", class_="devsite-article-body")
    for child in body_div.findChildren():
        if child.name == "p":
            paragraphs.append(child.get_text().strip())
        if child.name == "h2":
            sections.append(" ".join(paragraphs))
            break

    for header in soup.find_all("h2"):
        paragraphs = []
        nextNode = header.nextSibling
        while nextNode:
            if isinstance(nextNode, Tag):
                if nextNode.name in {"p", "ul"}:
                    paragraphs.append(nextNode.get_text().strip())
                elif nextNode.name == "h2":
                    sections.append(" ".join(paragraphs))
                    break
            nextNode = nextNode.nextSibling
    return sections

In [ ]:
all_text = [t for url in URLS for t in get_sections(url) if t]

In [ ]:
text_lengths = [len(t) for t in all_text]
pd.DataFrame(text_lengths).hist()